# Elevator Tracking

To track an elevator, 
- we extract the seven segment display on its display board.
- we convert it to a B&W binary image
- we can take two approaches to recognize the digits
 * a machine learning approach
 * an algorithmic approach (this works because the digits wont be handwritten, so their positions are fixed)
 
I found [an online IPTV](https://www.insecam.org/en/view/421702/) showing the interior of an elevator in Noord-Holland, Amsterdam.






The source video             |  A seven segment display
:-------------------------:|:-------------------------:
<img src="http://145.53.212.190:8001/mjpg/video.mjpg" width="60%">  |  <img src="https://www.direnc.net/Data/EditorFiles/aciklama-gorselleri-2/7-segment-display-ekran-pinout.jpg" width="60%">

In [ ]:
import cv2
import numpy as np

In [ ]:
chm = {"A": blackAndWhiteImage[10:14,25:38],
       "B": blackAndWhiteImage[15:26,36:40],
       "C": blackAndWhiteImage[25:37,35:38],
       "D": blackAndWhiteImage[36:41,21:33],
       "E": blackAndWhiteImage[25:37,18:22],
       "F": blackAndWhiteImage[14:25,19:23],
       "G": blackAndWhiteImage[21:26,22:36]}

arr = {"1111110": 0,
       "0110000": 1,
       "1101101": 2,
       "1111001": 3,
       "0110011": 4,
       "1011011": 5,
       "1011111": 6,
       "1110000": 7,
       "1111111": 8,
       "1111011": 9}

def active(X, threshold = 0.25):
    if np.average(X)/255 > threshold:
        return 1
    else:
        return 0

def getdigit(blackAndWhiteImage):
    
    chm = {"A": blackAndWhiteImage[10:15,25:38],
       "B": blackAndWhiteImage[15:26,36:40],
       "C": blackAndWhiteImage[25:37,35:38],
       "D": blackAndWhiteImage[36:41,21:33],
       "E": blackAndWhiteImage[25:37,18:22],
       "F": blackAndWhiteImage[14:25,19:23],
       "G": blackAndWhiteImage[21:26,22:36]}
    
    S = ""
    c = 0
    for key, val in chm.items():
        S += repr(active(val))
        c+=1
    return arr[S]

In [ ]:
cap = cv2.VideoCapture("http://145.53.212.190:8001/mjpg/video.mjpg")


#---- 4 corner points of the bounding box
pts_src = np.array([[123.0,53.0], [146.0,37.0], [129.0, 80.0],[158.0, 69.0]])

#---- 4 corner points of the black image you want to impose it on
pts_dst = np.array([[0.0,0.0],[50.0, 0.0],[ 0.0,50.0],[50.0, 50.0]])

#---- forming the black image of specific size
im_dst = np.zeros((50, 50, 3), np.uint8)

#---- Framing the homography matrix
h, status = cv2.findHomography(pts_src, pts_dst)


while True:
    ret, frame = cap.read()
    


    #---- transforming the image bound in the rectangle to straighten
    im_out = cv2.warpPerspective(frame, h, (im_dst.shape[1],im_dst.shape[0]))
    
    im_gray = cv2.cvtColor(im_out, cv2.COLOR_BGR2GRAY)
    
    (thresh, blackAndWhiteImage) = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY)

    digit = getdigit(blackAndWhiteImage)
    
    cv2.putText(
        im_out, #numpy array on which text is written
        repr(digit), #text
        (2,23), #position at which writing has to start
        cv2.FONT_HERSHEY_SIMPLEX, #font family
        1, #font size
        (255, 255, 0, 255), #font color
        2) #font stroke
        
    cv2.imshow('frame', im_out)


    if cv2.waitKey(1) & 0xff == ord('q'):
        break
        
# When everything done, release the capture
cap.release()
# and release the output
out.release()
# finally, close the window
cv2.destroyAllWindows()
cv2.waitKey(1)